In [1]:
import pandas as pd
import numpy as np
import plotly as pt
import plotly.express as px
import datetime as dt

In [2]:
cart = pd.read_csv("Downloads/adds to cart.csv")
sessions = pd.read_csv("Downloads/session counts.csv")

In [3]:
#DELIVERABLE 1 (Sheet 1)

In [4]:
#conversion process so that we may extract months and years
sessions.info()
sessions['dim_date']=pd.to_datetime(sessions['dim_date'])
#converting dataframe time to desired format, and extracting months/years
sessions['dim_year'] = pd.DatetimeIndex(sessions['dim_date']).year
sessions['dim_month'] = pd.DatetimeIndex(sessions['dim_date']).month

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7734 entries, 0 to 7733
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   dim_browser         7734 non-null   object
 1   dim_deviceCategory  7734 non-null   object
 2   dim_date            7734 non-null   object
 3   sessions            7734 non-null   int64 
 4   transactions        7734 non-null   int64 
 5   QTY                 7734 non-null   int64 
dtypes: int64(3), object(3)
memory usage: 362.7+ KB


In [5]:
#checking current format
sessions.head(10)

,dim_browser,dim_deviceCategory,dim_date,sessions,transactions,QTY,dim_year,dim_month
0,Safari,tablet,2012-07-01,2928,127,221,2012,7
1,Internet Explorer,desktop,2012-07-01,1106,28,0,2012,7
2,Chrome,tablet,2012-07-01,474,3,13,2012,7
3,Amazon Silk,tablet,2012-07-01,235,4,5,2012,7
4,Internet Explorer,mobile,2012-07-01,178,6,11,2012,7
5,Internet Explorer,tablet,2012-07-01,120,7,0,2012,7
6,Android Browser,mobile,2012-07-01,10,0,0,2012,7
7,error,desktop,2012-07-01,9,0,0,2012,7
8,Edge,mobile,2012-07-01,5,0,0,2012,7
9,Opera,mobile,2012-07-01,4,0,0,2012,7


In [6]:
#pivot table to aggregate metrics on monthly basis
sesh_pivot = sessions.pivot_table(
    values=["sessions", "transactions", "QTY"],
    index=["dim_year","dim_month","dim_deviceCategory"],
    aggfunc={'sessions':np.sum, 'transactions':np.sum, 'QTY':np.sum}
)


In [7]:
sesh_pivot.columns

Index(['QTY', 'sessions', 'transactions'], dtype='object')

In [8]:
#adding final KPI after aggregation to avoid averaging averages. allows for less distortion
sesh_pivot['ECR'] = sesh_pivot.transactions/sesh_pivot.sessions

In [9]:
#locking columns in proper order and checking work
sesh=sesh_pivot.loc[:, ["sessions", "transactions", "QTY","ECR"]]

sesh.head(10)


sessions  transactions    QTY       ECR
dim_year dim_month dim_deviceCategory                                         
2012     7         desktop               335429         10701  18547  0.031902
                   mobile                274443          2576   4557  0.009386
                   tablet                158717          4884   8700  0.030772
         8         desktop               392079         12912  23316  0.032932
                   mobile                275556          3165   5572  0.011486
                   tablet                154858          3202   5760  0.020677
         9         desktop               272771          8898  16507  0.032621
                   mobile                220689          2381   4050  0.010789
                   tablet                169193          4379   7869  0.025882
         10        desktop               302682          9373  17675  0.030966

In [11]:
#END OF DELIVERABLE 1, BEGINNING OF DELIVERABLE 2

In [12]:
#creating first pivot table that will eventually be paired with another pivot table

cart.head(10)

df1 = pd.DataFrame(cart.pivot_table(
    values=["addsToCart"],
    index=["dim_year","dim_month"],
    aggfunc={'addsToCart':np.sum}))


df1.head(10)


addsToCart
dim_year dim_month            
2012     7              191504
         8              217666
         9              123726
         10             139803
         11             186572
         12             168972
2013     1              147619
         2              135882
         3              109797
         4              183842

In [13]:
#second pivot table in which the first will be joined onto

df2 = pd.DataFrame(sessions.pivot_table(
    values=["sessions","transactions","QTY"],
    index=["dim_year","dim_month"],
    aggfunc={'sessions':np.sum,
             'transactions':np.sum,
             'QTY':np.sum}
             ))
df2.head(10)

QTY  sessions  transactions
dim_year dim_month                               
2012     7          31804    768589         18161
         8          34648    822493         19279
         9          28426    662653         15658
         10         26626    648639         14275
         11         28132    637780         15527
         12         34752    789634         19929
2013     1          38846    899992         21560
         2          27048    550227         14166
         3          32082    788820         17804
         4          54946   1296613         30369

In [14]:
#function used to merge the data in both tables
all_data = pd.merge(df1, df2, 'left', on = ["dim_year", "dim_month"] )

In [15]:
#defining percentage function that will help create columns with relative differences
def percentage_change(col1,col2):
    return ((col2/(col1-col2))*100) 

In [16]:
#creation of columns that compare month to month data
all_data['adds_OTM'] = all_data.addsToCart.diff()
all_data['adds_OTM_%'] = percentage_change(all_data['addsToCart'],all_data['adds_OTM'])
all_data['QTY_OTM'] = all_data.QTY.diff()
all_data['QTY_OTM_%'] = percentage_change(all_data['QTY'],all_data['QTY_OTM'])
all_data['ECR'] = all_data.transactions/all_data.sessions
all_data['ECR_OTM'] = all_data.ECR.diff() 
all_data['ECR_OTM_%'] = percentage_change(all_data['ECR'],all_data['ECR_OTM'])
all_data['sessions_OTM'] = all_data.sessions.diff()
all_data['sessions_OTM_%'] = percentage_change(all_data['sessions'],all_data['sessions_OTM'])
all_data['transactions_OTM'] = all_data.transactions.diff()
all_data['transactions_OTM_%'] = percentage_change(all_data['transactions'],all_data['QTY_OTM'])



In [17]:
#locking columns in proper order
all_data=all_data.loc[:, ["addsToCart", "adds_OTM","adds_OTM_%",
                          "QTY", "QTY_OTM", "QTY_OTM_%",
                          "ECR", "ECR_OTM", "ECR_OTM_%",
                          "sessions", "sessions_OTM", "sessions_OTM_%",
                          "transactions", "transactions_OTM", "transactions_OTM_%"]]

In [18]:
#creating final sheet with only the desirable months of data
final = all_data[all_data.index.get_level_values('dim_month').isin(['3', '4'])]

In [19]:
final.head(10)

addsToCart  adds_OTM  adds_OTM_%    QTY  QTY_OTM  \
dim_year dim_month                                                     
2013     3              109797  -26085.0  -19.196803  32082   5034.0   
         4              183842   74045.0   67.438090  54946  22864.0   

                    QTY_OTM_%       ECR   ECR_OTM  ECR_OTM_%  sessions  \
dim_year dim_month                                                       
2013     3          18.611358  0.022570 -0.003175 -12.333366    788820   
         4          71.267377  0.023422  0.000851   3.772062   1296613   

                    sessions_OTM  sessions_OTM_%  transactions  \
dim_year dim_month                                               
2013     3              238593.0       43.362649         17804   
         4              507793.0       64.373748         30369   

                    transactions_OTM  transactions_OTM_%  
dim_year dim_month                                        
2013     3                    3638.0           39.420517  
         4                   12565.0          304.650233

In [30]:
#exporting both sheets into 1 excel file
writer = pd.ExcelWriter("Downloads/IXIS_FINAL.xlsx")
# Write each dataframe to a different worksheet.
sesh.to_excel(writer, sheet_name="sheet1", index=True)
final.to_excel(writer, sheet_name="sheet2", index=True)
writer.save()

In [21]:
#END of Deliverable 2